In [7]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S

from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(1)

In [2]:
print(os.listdir("image2"))

['sadness', 'anger', 'love', 'fear', 'surprise', 'joy']


In [3]:
images_dir = "image2"
emotion_dirs = os.listdir("image2")
emotion_dirs

['sadness', 'anger', 'love', 'fear', 'surprise', 'joy']

In [4]:
weight = {'anger' : 440, 'fear': 153, 'joy':587, 'love': 650, 'sadness': 758, 'surprise': 457}

In [5]:
class_weights = {}
for i in range(6):
    w = 3045 / (6 * weight[emotion_dirs[i]])
    class_weights[i] = w 
class_weights

{0: 0.6695250659630607,
 1: 1.1534090909090908,
 2: 0.7807692307692308,
 3: 3.316993464052288,
 4: 1.1105032822757113,
 5: 0.8645655877342419}

In [6]:
batch_size = 64
train_input_shape = (224, 224, 3)
n_classes = 6

train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                    classes=emotion_dirs
                                                   )

valid_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                    classes=emotion_dirs
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)

Found 2438 images belonging to 6 classes.
Found 607 images belonging to 6 classes.
Total number of batches = 38 and 9


In [8]:
# Load pre-trained model
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=train_input_shape)

for layer in base_model.layers:
    layer.trainable = True

2022-10-25 03:56:38.269769: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 03:56:38.270969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 03:56:38.271731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 03:56:38.272694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 03:56:38.278004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

82420632/82420632 [==============================] - 2s 0us/step


In [9]:
# Add layers at the end
X = base_model.output
X = Flatten()(X)

X = Dense(512, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(16, kernel_initializer='he_uniform')(X)
#X = Dropout(0.5)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

output = Dense(n_classes, activation='softmax')(X)

model = Model(inputs=base_model.input, outputs=output)

In [10]:
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

In [11]:
n_epoch = 20

early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                              verbose=1, mode='auto')

In [12]:
# Train the model - all layers
history1 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr],
                              use_multiprocessing=True,
                              workers=16,
                              class_weight=class_weights
                             )

/tmp/ipykernel_3506050/4005977698.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history1 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,


Epoch 1/20
38/38 [==============================] - 404s 10s/step - loss: 1.6941 - accuracy: 0.2485 - val_loss: 2.7080 - val_accuracy: 0.0972 - lr: 1.0000e-04
Epoch 2/20
38/38 [==============================] - 409s 9s/step - loss: 1.3095 - accuracy: 0.4368 - val_loss: 2.4589 - val_accuracy: 0.2396 - lr: 1.0000e-04
Epoch 3/20
38/38 [==============================] - 427s 10s/step - loss: 1.1024 - accuracy: 0.5695 - val_loss: 2.1217 - val_accuracy: 0.3021 - lr: 1.0000e-04
Epoch 4/20
38/38 [==============================] - 447s 10s/step - loss: 0.9477 - accuracy: 0.6559 - val_loss: 1.9764 - val_accuracy: 0.3073 - lr: 1.0000e-04
Epoch 5/20
38/38 [==============================] - 439s 10s/step - loss: 0.8380 - accuracy: 0.7266 - val_loss: 2.0385 - val_accuracy: 0.3403 - lr: 1.0000e-04
Epoch 6/20
38/38 [==============================] - 455s 10s/step - loss: 0.7353 - accuracy: 0.7869 - val_loss: 1.9783 - val_accuracy: 0.3264 - lr: 1.0000e-04
Epoch 7/20
38/38 [=============================

In [ ]:
# Freeze core ResNet layers and train again 
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[:50]:
    layer.trainable = True

optimizer = Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

n_epoch = 50
history2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr, early_stop],
                              use_multiprocessing=True,
                              workers=16,
                              class_weight=class_weights
                             )

/home/jupyter-k7a506/.local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_3506050/2442743920.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,


Epoch 1/50
38/38 [==============================] - 490s 11s/step - loss: 0.2785 - accuracy: 0.9861 - val_loss: 1.8682 - val_accuracy: 0.3351 - lr: 1.0000e-04
Epoch 2/50
38/38 [==============================] - 504s 12s/step - loss: 0.2687 - accuracy: 0.9857 - val_loss: 1.9003 - val_accuracy: 0.3507 - lr: 1.0000e-04
Epoch 3/50
38/38 [==============================] - 510s 12s/step - loss: 0.2623 - accuracy: 0.9836 - val_loss: 1.8585 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 4/50
38/38 [==============================] - 473s 11s/step - loss: 0.2623 - accuracy: 0.9886 - val_loss: 1.8239 - val_accuracy: 0.3594 - lr: 1.0000e-04
Epoch 5/50
38/38 [==============================] - 526s 12s/step - loss: 0.2632 - accuracy: 0.9865 - val_loss: 1.8818 - val_accuracy: 0.3281 - lr: 1.0000e-04
Epoch 6/50
38/38 [==============================] - 497s 11s/step - loss: 0.2593 - accuracy: 0.9848 - val_loss: 1.8604 - val_accuracy: 0.3559 - lr: 1.0000e-04
Epoch 7/50
38/38 [============================

In [ ]:
model.save('efficientnet_v2_s.h5')